In [1]:
import os, sys
%env LD_LIBRARY_PATH=/usr/local/cuda-8.0/lib64
%env LIBRARY_PATH=/usr/local/cuda-8.0/lib64
%env PATH=/usr/local/cuda-8.0/include:/usr/local/cuda-8.0/bin:/home/apanin/anaconda/bin:/opt/WinCC_OA/3.11/bin:/opt/pvss/pvss2_v3.8/bin:/home/apanin/anaconda/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/puppetlabs/bin:/opt/fmc/bin:/group/online/bin:/group/online/scripts:/home/apanin/bin:/home/apanin/bin
sys.path.insert(0,"/home/apanin/rit_my/")
%env CUDA_VISIBLE_DEVICES=0
%env MKL_THREADING_LAYER=GNU
%env THEANO_FLAGS=device=cpu,gpuarray.preallocate=0.99


env: LD_LIBRARY_PATH=/usr/local/cuda-8.0/lib64
env: LIBRARY_PATH=/usr/local/cuda-8.0/lib64
env: PATH=/usr/local/cuda-8.0/include:/usr/local/cuda-8.0/bin:/home/apanin/anaconda/bin:/opt/WinCC_OA/3.11/bin:/opt/pvss/pvss2_v3.8/bin:/home/apanin/anaconda/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/puppetlabs/bin:/opt/fmc/bin:/group/online/bin:/group/online/scripts:/home/apanin/bin:/home/apanin/bin
env: CUDA_VISIBLE_DEVICES=0
env: MKL_THREADING_LAYER=GNU
env: THEANO_FLAGS=device=cpu,gpuarray.preallocate=0.99


In [2]:
import theano
import theano.tensor as T
import lasagne
import lasagne.layers as L

## preprocess

In [3]:
!./preprocess.sh data/indomain_training/indomain.de-en.de preprocessed/ en de
!./preprocess.sh data/indomain_training/indomain.de-en.en preprocessed/ de en

Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: de
Number of threads: 1


In [3]:
SRC_PATH = "data/wmt/train.tok.bpe.32000.en" 
DST_PATH = "output.txt" 

In [4]:
raw_src = []
with open(SRC_PATH, 'r') as f:
    raw_src = f.readlines()

In [5]:
raw_src = raw_src[:1000]

In [6]:
def fix_ends(sents):
    for i in range(len(sents)):
        sents[i] = sents[i][:-2]

In [7]:
fix_ends(raw_src)

In [9]:
class Vocab:
    def __init__(self, sentences):
        tokens = set()
        for s in sentences:
            tokens.update(s.split(' '))
        self.tokens = [ "__BOS__","__EOS__", "__PAD__"] + list(tokens)
        self.EOS = 1
        self.BOS = 0 # BOS should be zero to let the model generate starting with zero as an input
        self.PAD = 2
        self.len = len(self.tokens)
        self.token2id = {token: i for i, token in enumerate(self.tokens)}
    def tokenize(self, sentence):
        if not sentence.endswith("__EOS__"):
            sentence += " __EOS__"
        if not sentence.startswith("__BOS__"):
            sentence = "__BOS__ " + sentence
        return [self.token2id[token] for token in sentence.split(' ')]
    def detokenize(self, sentence):
        return " ".join([self.tokens[token] for token in sentence])
    def tokenize_many(self, sentences):
        return [self.tokenize(sent) for sent in sentences]
    def detokenize_many(self, sentences):
        return [self.detokenize(sent) for sent in sentences]

In [10]:
import pickle as pkl
with open("vocabs.pkl", 'rb') as f:
    dst_voc = pkl.load(f)
    src_voc = pkl.load(f)

In [11]:
X = src_voc.tokenize_many(raw_src)

In [12]:
MAX_LEN = 100

In [13]:
import numpy as np

In [14]:
np.random.seed(42)

In [15]:
%env KERAS_BACKEND=theano
from keras.preprocessing.sequence import pad_sequences

env: KERAS_BACKEND=theano


Using Theano backend.


In [16]:
input_sequence = T.matrix('token sequencea','int32')
input_mask = T.neq(input_sequence, src_voc.PAD)

target_values = T.matrix('actual next token','int32')
target_mask = T.neq(target_values, dst_voc.PAD)

In [17]:
CODE_SIZE = 512

In [18]:
l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)
l_mask = lasagne.layers.InputLayer(shape=(None, None),input_var=input_mask)

#encoder
l_emb = L.EmbeddingLayer(l_in, src_voc.len, 128)

l_rnn = L.LSTMLayer(l_emb, 256, nonlinearity=T.tanh, mask_input= l_mask)
l_rnn = L.concat([l_emb,l_rnn], axis=-1)
l_encoded = l_rnn = L.LSTMLayer(l_rnn, CODE_SIZE, nonlinearity=T.tanh, mask_input= l_mask)

l_trans = L.InputLayer((None, None), input_var= target_values[:,:-1])
l_trans_mask = L.InputLayer((None, None), input_var= target_mask[:,:-1])


In [19]:
from agentnet.agent.recurrence import Recurrence
from agentnet.memory import AttentionLayer,LSTMCell 

In [20]:
from agentnet.resolver import ProbabilisticResolver, GreedyResolver

In [21]:
class AutoLSTMCell:
    def __init__(self, input_or_inputs, num_units = None, *args, name=None, **kwargs):
        self.p_cell = L.InputLayer((None, num_units), 
                       name="previous cell state" if name == None else name + " previous cell state")
        self.p_out = L.InputLayer((None, num_units), 
                       name="previous out state" if name == None else name + " previous out state")
        self.cell, self.out = LSTMCell(self.p_cell, self.p_out, input_or_inputs, num_units, *args,name=name, **kwargs)
    def get_automatic_updates(self):
        return {self.cell: self.p_cell, self.out: self.p_out}

In [22]:
class TemperatureResolver(ProbabilisticResolver):
    def __init__(self, incoming, tau, **kwargs):
        self.tau = tau
        super(TemperatureResolver, self).__init__(incoming,**kwargs)
    def get_output_for(self, policy, **kwargs):
        policy = policy ** (1/self.tau)
        policy /= policy.sum()
        return super(TemperatureResolver, self).get_output_for(policy, **kwargs) 

In [23]:
class decoder_step:
    #inputs
    encoder = L.InputLayer((None, None ,CODE_SIZE), name='encoded sequence')
    encoder_mask = L.InputLayer((None, None), name='encoded sequence')
    
    inp = L.InputLayer((None,),name='current character')
    
    l_target_emb = L.EmbeddingLayer(inp, dst_voc.len, 128)
    
    #recurrent part
    
    l_rnn1 = AutoLSTMCell(l_target_emb, 128, name="lstm1")
    
    query = L.DenseLayer(l_rnn1.out, 128, nonlinearity=None)
    attn = AttentionLayer(encoder, query, 128, mask_input= encoder_mask)['attn']
    
    l_rnn = L.concat([attn, l_rnn1.out, l_target_emb])
    
    l_rnn2 = AutoLSTMCell(l_rnn, 128, name="lstm1")
    
    next_token_probas = L.DenseLayer(l_rnn2.out, dst_voc.len, nonlinearity=T.nnet.softmax)
    
    #pick next token from predicted probas
    next_token = ProbabilisticResolver(next_token_probas)
    
    tau = T.scalar("sample temperature", "float32")
    
    next_token_temperatured = TemperatureResolver(next_token_probas, tau)
    next_token_greedy = GreedyResolver(next_token_probas)
    
    auto_updates = {**l_rnn1.get_automatic_updates(),
                    **l_rnn2.get_automatic_updates()}

In [24]:
from collections import OrderedDict

In [25]:
n_steps = T.scalar(dtype='int32')
feedback_loop = Recurrence(
    state_variables=OrderedDict({**decoder_step.auto_updates,
                     decoder_step.next_token:decoder_step.inp}),
    tracked_outputs=[decoder_step.next_token_probas, decoder_step.next_token],
    input_nonsequences= OrderedDict({decoder_step.encoder: l_encoded, decoder_step.encoder_mask: l_mask} ),
    batch_size=input_sequence.shape[0],
    n_steps=n_steps,
    unroll_scan=False,
)

In [26]:
# Model weights
weights = lasagne.layers.get_all_params(feedback_loop,trainable=True)
print (weights)

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, lstm1.b_to_ingate, lstm1.W_lstm1 previous out state_to_ingate, lstm1.W_ctrl1_to_ingate, lstm1.b_to_forgetgate, lstm1.W_lstm1 previous out state_to_forgetgate, lstm1.W_ctrl1_to_forgetgate, lstm1.b_to_cell, lstm1.W_lstm1 previous out state_to_cell, lstm1.W_ctrl1_to_cell, lstm1.b_to_outgate, lstm1.W_lstm1 previous out state_to_outgate, lstm1.W_ctrl1_to_outgate, lstm1.W_cell_to_ingate_peephole.scales, lstm1.W_cell_to_forgetgate_peephole.scales, lstm1.W_cell_to_outgate_peephole.scales, W, b, enc_to_hid, dec_to_hi

In [27]:
generated_tokens = L.get_output(feedback_loop[decoder_step.next_token])

generate_sample = theano.function([input_sequence ,n_steps],generated_tokens,
                                  updates=feedback_loop.get_automatic_updates())

In [28]:
feedback_loop_greedy = Recurrence(
    state_variables=OrderedDict({**decoder_step.auto_updates,
                     decoder_step.next_token_greedy:decoder_step.inp}),
    tracked_outputs=[decoder_step.next_token_probas, decoder_step.next_token_greedy],
    input_nonsequences= OrderedDict({decoder_step.encoder: l_encoded, decoder_step.encoder_mask: l_mask} ),
    batch_size=input_sequence.shape[0],
    n_steps=n_steps,
    unroll_scan=False,
)

In [29]:
generated_tokens_greedy = L.get_output(feedback_loop_greedy[decoder_step.next_token_greedy])

generate_sample_greedy = theano.function([input_sequence ,n_steps],generated_tokens_greedy,
                                  updates=feedback_loop_greedy.get_automatic_updates())

In [30]:
feedback_loop_temp = Recurrence(
    state_variables=OrderedDict({**decoder_step.auto_updates,
                     decoder_step.next_token_temperatured:decoder_step.inp}),
    tracked_outputs=[decoder_step.next_token_probas, decoder_step.next_token_temperatured],
    input_nonsequences= OrderedDict({decoder_step.encoder: l_encoded, decoder_step.encoder_mask: l_mask} ),
    batch_size=input_sequence.shape[0],
    n_steps=n_steps,
    unroll_scan=False,
)

In [31]:
generated_tokens_temp = L.get_output(feedback_loop_temp[decoder_step.next_token_temperatured])

generate_sample_temp = theano.function([input_sequence ,n_steps, decoder_step.tau],generated_tokens_temp,
                                  updates=feedback_loop_temp.get_automatic_updates())

In [33]:
weights_dict = {str(i): weight for i, weight in enumerate(weights)}

In [34]:
with np.load("weights.npz") as f:
    for key in weights_dict:
        weights_dict[key].set_value(f[key])

In [35]:
from tqdm import tqdm

In [36]:
from batch_iterator import iterate_minibatches

In [61]:
def translate(srcs, N=MAX_LEN,t=1, greedy=False):
    srcs = [src_voc.tokenize(src) for src in srcs]
    if len(srcs) > 1:
        srcs = pad_sequences(srcs, value= src_voc.PAD, maxlen=MAX_LEN, padding="post")
    sample_ix = generate_sample_greedy(srcs, N) if greedy else generate_sample_temp(srcs, N, t)
    random_snippet = dst_voc.detokenize_many(sample_ix)
    res = []
    for sent in random_snippet:
        if sent.find("__EOS__") > 0:
            res.append(sent[:sent.find("__EOS__")].replace("@@ ", ""))
        
    return res

In [86]:

with open(DST_PATH, 'w') as f:
    for start in tqdm(range(0, len(raw_src), 1)):
        f.writelines(translate(raw_src[start: start+1], t=0.2))


  0%|          | 3/1000 [00:02<12:59,  1.28it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/apanin/anaconda/envs/py35/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/apanin/anaconda/envs/py35/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/apanin/anaconda/envs/py35/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 54%|█████▎    | 536/1000 [06:13<05:23,  1.43it/s]


KeyboardInterrupt: 

In [79]:
translating_ind = np.random.randint(len(raw_src))

In [80]:
print(raw_src[translating_ind])
translate([raw_src[translating_ind]], t=0.1, greedy=False)

This is not just the fault of the Commission , but I believe that we need to take action more quickly so as to achieve harmonisation in this area as well 


['Das ist nicht nur die Kommission , aber ich glaube , dass wir uns auf die Notwendigkeit , die wir in diesem Bereich zu erreichen , um die Lösung zu gewährleisten , um die Lösung zu gewährleisten  ']